In [1]:
from google.cloud import bigquery
from google.oauth2 import service_account
from pandas.io import gbq
import pandas_gbq

In [2]:
# These first two values will be different on your machine. 
service_path = "/Users/liamhettinger/Documents/Fall_MSBA/Applied_Data_Analytics/wedge/"
service_file = 'fleet-geode-327423-3bd2c958746e.json' # change this to your authentication information  
gbq_proj_id = 'fleet-geode-327423' # change this to your poroject. 
gbq_dataset_id = 'Wedge'

# And this should stay the same. 
private_key = service_path + service_file

In [3]:
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

In [4]:
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [5]:
sql = """
SELECT DISTINCT card_no
FROM `fleet-geode-327423.Wedge.transArchive_*` 
where NOT card_no = 3 
"""
owner_list = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(owner_list, 'Wedge.Owner_list', project_id=gbq_proj_id, if_exists='replace')

Downloading: 100%|██████████| 27208/27208 [00:00<00:00, 35348.28rows/s]
1it [00:03,  3.36s/it]


In [6]:
len(owner_list) * .01

272.08

In [7]:
sql = """
SELECT DISTINCT card_no
FROM `fleet-geode-327423.Wedge.transArchive_*`
where NOT card_no = 3 
ORDER BY RAND()
LIMIT 280
"""
Sample_owner_list = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(Sample_owner_list, 'Wedge.Sample_Owner_list', project_id=gbq_proj_id, if_exists='replace')

Downloading: 100%|██████████| 280/280 [00:00<00:00, 1215.45rows/s]
1it [00:03,  3.03s/it]


In [8]:
sql = """
    SELECT *
    FROM `fleet-geode-327423.Wedge.transArchive_*`
    WHERE card_no IN
    (SELECT card_no 
     FROM `fleet-geode-327423.Wedge.Sample_Owner_list`)
"""
All_data_sample_owners = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(All_data_sample_owners, 'Wedge.All_data_sample_owners', project_id=gbq_proj_id, if_exists='replace')

Downloading: 100%|██████████| 565072/565072 [03:31<00:00, 2672.23rows/s]
1it [02:55, 175.68s/it]


Error is from trying to send it back to Google big query. Don't want to run it again because it takes so long. Still was able to create csv file

In [9]:
All_data_sample_owners.to_csv('owners_sample.csv')

In [ ]:
# Select * error
sql = """
    "SELECT * "
    "FROM `fleet-geode-327423.Wedge.transArchive_*` "
    "WHERE card_no IN (
    SELECT card_no 
    FROM `fleet-geode-327423.Wedge.Sample_Owner_list`
    )
"""
All_data_sample_owners = pandas_gbq.read_gbq(sql, project_id=gbq_proj_id)
pandas_gbq.to_gbq(All_data_sample_owners, 'Wedge.All_data_sample_owners', project_id=gbq_proj_id, if_exists='replace')

In [ ]:
table_ref = client.get_table('fleet-geode-327423.Wedge.All_data_sample_owners')

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
SELECT
  *
EXCEPT (tax,datetime)
FROM
  `fleet-geode-327423.Wedge.transArchive_*`
WHERE
  card_no IN (
  SELECT
    *
  FROM
    `fleet-geode-327423.Wedge.Sample_Owner_list`)
"""



query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))